# Ex07 - Propriedades da DFT

### Parte 1 - Convolução periódica

Tente entender a convolução periódica [(notebook)](11 Teorema da Convolucao.ipynb), comparando-a com a convolução já estudada anteriormente (scypi.signal.convolve2d). Avalie as diferenças em um exemplo numérico pequeno e depois, utilizando uma imagem.

- Desafio opcional: crie um exemplo com uma imagem numérica pequena e um kernel 3x3. Tente obter o mesmo resultado usando a função *convp* e a função *scypi.signal.convolve2d*. Ou seja, implemente a convolução periódica a partir da convolução linear.

**Fernando:** 
Para chegar ao mesmo resultado do pconv, eu utilizei a scipy.signal.concolve2d no modo full, com os limites do tipo wrap e formei a matriz de resultado a partir da origem dessa nova matriz, do tamanho da imagem original.

In [ ]:
import sys,os
ia898path = os.path.abspath('../../')
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

from matplotlib import image as mpimg
from matplotlib import pyplot as plt
import numpy as np
from numpy.fft import fftshift
from scipy.signal import convolve2d

#### Com uma matriz numérica simples (a mesma do exempo)

In [ ]:
f_p1 = np.array([[1,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0],
             [0,0,0,1,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,1],
             [0,0,0,0,0,0,0,0,0]]).astype('float32')

kernel_p1a = np.arange(1,10,1).reshape((3,3))

print(f_p1, f_p1.shape)

g_p1a = convolve2d(f_p1, kernel_p1a, mode='full', boundary='wrap')
g_p1a = g_p1a[:f_p1.shape[0],:f_p1.shape[1]]
g_p1b = ia.pconv(f_p1, kernel_p1a)

print(g_p1a, g_p1a.shape)

print(g_p1b, g_p1b.shape)

In [ ]:
print(kernel_p1a.shape)
print(f_p1.shape)

pad = tuple(np.subtract(f_p1.shape, kernel_p1a.shape)//2)
print(pad)

**Fernando:** Implementando com uma imagem, com filtro de sobel

In [ ]:
f_p1a = np.array(mpimg.imread('../data/cameraman.tif'))

kernel_p1c = np.array([[1, 0, -1],
                      [2, 0, -2],
                      [1, 0, -1]])

g_p1c = convolve2d(f_p1a, kernel_p1c, mode='full', boundary='wrap')
g_p1c = g_p1c[:f_p1a.shape[0],:f_p1a.shape[1]]

g_p1d = ia.pconv(f_p1a, kernel_p1c)

g_p1e = convolve2d(f_p1a, kernel_p1c, mode='same')
                 
fig = plt.figure(figsize=[15,15])
fig.add_subplot(321)
plt.title('0: Imagem original')
plt.imshow(f_p1a, cmap='gray')

fig.add_subplot(323)
plt.title('1: Utilizando convolve2d')
plt.imshow(g_p1c, cmap='gray')

fig.add_subplot(324)
plt.title('2: Utilizando pconv')
plt.imshow(g_p1d, cmap='gray')

fig.add_subplot(325)
plt.title('3: convolve2d, modo same, sem periodico')
plt.imshow(g_p1e, cmap='gray')

plt.show()

print('numpy.sum da diferença entre as duas imagens 1 e 2: {}'.format(np.sum(g_p1c - g_p1d)))
print('numpy.sum da diferença entre as duas imagens 2 e 3: {}'.format(np.sum(g_p1d - g_p1e)))

### Parte 2 - Teorema da convolução

Crie uma demonstração usando imagens para o Teorema da Convolução. Lembre-se, o objetivo é mostrar que o teorema é válido, ou seja, que $ F(f * g) = F(f) \cdot F(g) $

**Fernando:** Criando a imagem e os kernels.

In [ ]:
from numpy.fft import fft2, ifft2

f_p2a = np.array(mpimg.imread('../data/cameraman.tif'))

kernel_p2a = np.array([[1, 0, -1],
                      [2, 0, -2],
                      [1, 0, -1]])

r, c = kernel_p2a.shape
k2a_aux = np.zeros(f_p2a.shape)
k2a_aux[:r, :c] = kernel_p2a

**Fernando:** Fazendo as transformadas e aplicando filtros

In [ ]:
F_p2a = np.fft.fft2(f_p2a)
K_p2a = np.fft.fft2(k2a_aux)

Multiplicação no dominio das frequências

In [ ]:
G_p2a = F_p2a * K_p2a
g_p2a = np.fft.ifft2(G_p2a)

plt.imshow(g_p2a.real, cmap='gray')
plt.show()

Convolução no domínio do espaço e transformada de fourier do resultado

In [ ]:
g_p2b = ia.pconv(f_p2a, kernel_p2a)

plt.imshow(g_p2b.real, cmap='gray')
plt.show()

G_p2b = np.fft.fft2(g_p2b)

Comparação com numpy.allclose

In [ ]:
print(np.allclose(G_p2a, G_p2b))

### Parte3 - Decomposição de uma imagem em senóides

Implemente uma função para demonstrar a decomposição/composição de uma imagem a partir da DFT. A ideia consiste em ter 4 figuras por iteração: 

- imagem original, 
- imagem recomposta até aquela iteração, 
- espectro de Fourier, e 
- última componente ("telha") a ser adicionada. 

A medida que um novo par *F(u,v)* e *F(-u,-v)* em cada iteração é adicionado para compor a imagem original, o par deve ser mostrado na janela do espectro de Fourier e na forma de imagem ("telha"). A ordem de pegar estas frequências pode ser uma varredura quadrada em torno do centro do espectro. Lembrar de pegar sempre um par simétrico da DFT, *F(u,v)* e *F(-u,-v)*. A função deverá receber como parâmetro de entrada quantas iterações serão realizadas, ou seja, quantas componentes serão adicionadas para compor a imagem original. 

In [ ]:
def shift_slice(img, rr, cc):
    
    f = np.array(img)
    r, c = f.shape

    c1 = cc%c
    r1 = rr%r
    g = np.concatenate(( np.concatenate((f[r-r1:,c-c1:], f[r-r1:,:c-c1]), axis=1) , \
                         np.concatenate((f[:r-r1, c-c1:], f[:r-r1, :c-c1]), axis=1) ))

    return g

In [ ]:
def show_fourier(F):
    F_final = ia.normalize(np.log(np.abs(shift_slice(F,F.shape[0]//2,F.shape[1]//2)+1)))
    return F_final

In [ ]:
from matplotlib import image as mpimg
from matplotlib import pyplot as plt
import numpy as np
from numpy.fft import fft2,ifft2

def decomposition(img, iterations):
    
    count = 0
    f = np.array(img)
    
    F_final = np.zeros(f.shape, dtype=complex)
    
    F = np.fft.fft2(f)
    
    F_aux = np.zeros(f.shape, dtype=complex)
    
    if iterations <= len(f.ravel())//2:
        N = iterations
    else:
        N = len(f.ravel())//2

    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            F_aux = np.zeros(f.shape, dtype=complex)
            F_aux[i, j] = F[i, j]
            F_aux[-(i+1),-(j+1)] =  F[-(i+1),-(j+1)]
            F_final += F_aux
            
            count +=1
            
            if count == N:
                break
        if count == N:
            break
    f_aux = np.fft.ifft2(F_aux)

    f_final = np.fft.ifft2(F_final)

    f3 = np.fft.ifft2(F)

    fig = plt.figure(figsize=[10, 10])
    fig.add_subplot(221)
    plt.imshow(np.abs(f), cmap='gray')

    fig.add_subplot(222)
    plt.imshow(np.abs(f_final), cmap='gray')
    
    fig.add_subplot(223)
    plt.imshow(show_fourier(F_final), cmap='gray')
    
    fig.add_subplot(224)
    plt.imshow(np.abs(f_aux), cmap='gray')
    plt.show()

Ao invés da janela quadrada, eu fiz a varredura começando da origem, mas somente incrementando os índices. Como fiz sem transladar a imagem no domínio das frequências, bastou começar no canto superior direito. Defini para cobrir somente metade da imagem, já que os pontos são selecionados sempre simétrico-conjugadamente.

In [ ]:
import os, sys
ia898path = os.path.abspath('../../')
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

from matplotlib import image as mpimg
from matplotlib import pyplot as plt
import numpy as np
from numpy.fft import fft2,ifft2

f1 = mpimg.imread('../data/cameraman.tif')

decomposition(f1, 6000)